In [1]:
using OrdinaryDiffEq, MLToolkit.Plots

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /Users/colehurwitz/.julia/packages/CUDAdrv/b1mvw/src/CUDAdrv.jl:67


In [2]:
abstract type AbstractObject end

struct Particle{R<:Real, V<:AbstractVector{R}} <: AbstractObject
    mass::R
    position::V
    velocity::V
end

massof(p::Particle) = p.mass
positionof(p::Particle) = p.position
velocityof(p::Particle) = p.velocity

###

abstract type AbstractForce end

struct Force{V<:AbstractVector{<:Real}} <: AbstractForce
    F::V
end

forceof(f::Force) = f.F

###

abstract type AbstractEnvironment end

struct Earth{F<:AbstractForce} <: AbstractEnvironment
    force::F
end

getforce(earth::Earth, p::Particle) = forceof(earth.force) + massof(p) * [0.0, -9.8]

getforce (generic function with 1 method)

In [3]:
function step(obj::T, env; dt=0.1) where {T<:Particle}
    # d position d t = velocity
    dpdt(pos, vel, p, t) = vel
    # d velocity d t = acceleration
    dvdt(pos, vel, p, t) = getforce(env, obj) / massof(obj)

    problem = DynamicalODEProblem(dpdt, dvdt, positionof(obj), velocityof(obj), (0.0, 1.0))
    integrator = init(problem, VerletLeapfrog(); dt=dt)
    step!(integrator, dt)
    
    return T(massof(obj), integrator.u.x[1], integrator.u.x[2])
end

function steps(obj::T, env, n_steps; dt=0.1) where {T<:Particle}
    path = Vector{T}(undef, n_steps)
    for i in 1:n_steps
        path[i] = step(obj, env; dt=dt)
        obj = path[i]
    end
    return path
end

obj = Particle(1.0, [0.0, 0.0], [10.0, 20.0])
env = Earth(Force([0.0, -10.0]))

traj = steps(obj, env, 10)

;

In [4]:
function plot(obj, env)
    traj = steps(obj, env, 10)

    pos = hcat(positionof.(traj)...)
    vel = hcat(velocityof.(traj)...)
    spd = sqrt.(dropdims(sum(vel.^2; dims=1); dims=1))

    ncols = 3
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(ncols * 5, 5))
    let ax = axes[1]
        plot!(ax, TwoDimPath(pos), "-o"; first=(marker="x", color="black"))
        ax.set_title("Position")
        ax.axis("equal")
    end
    let ax = axes[2]
        plot!(ax, TwoDimPath(vel), "-o"; first=(marker="x", color="black"))
        ax.set_title("Velocity")
        ax.axis("equal")
    end

    let ax = axes[3]
        ax.plot(spd, "-o")
        ax.set_title("Speed")
    end
    
    return fig
end

plot_data() = plot(obj, env)

plot_data() |> display

;

UndefVarError: UndefVarError: TwoDimPath not defined

In [5]:
using Turing

In [6]:
@model single_force(pos_list, vel_list) = begin
    F ~ MvNormal(zeros(2), 10 * ones(2))
    
    obj = Particle(1.0, [0.0, 0.0], [10.0, 20.0])
    env = Earth(Force(F))
    
    for i in 1:length(pos_list)
        obj′ = step(obj, env; dt=0.1)
        pos_list[i] ~ MvNormal(positionof(obj′), 0.1)
        vel_list[i] = vel_list[i]
        obj = obj′
    end
end
chain = sample(single_force(positionof.(traj), velocityof.(traj)), PG(100), 500)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:29


Object of type Chains, with data of type 500×5×1 Array{Float64,3}

Log evidence      = 20.713216278172762
Iterations        = 1:500
Thinning interval = 1
Chains            = 1
Samples per chain = 500
internals         = le, lp, weight
parameters        = F[1], F[2]

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse      │ ess     │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64   │ Any     │
├─────┼────────────┼─────────┼──────────┼────────────┼───────────┼─────────┤
│ 1   │ F[1]       │ 0.18467 │ 0.474368 │ 0.0212144  │ 0.0502233 │ 6.84746 │
│ 2   │ F[2]       │ -10.113 │ 0.149285 │ 0.00667621 │ 0.0367561 │ 4.38726 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%    │ 50.0%    │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64   │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼────────────┼───────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ F[1]       

In [16]:
positionof.(traj)[2]

2-element Array{Float64,1}:
 2.0  
 3.604

In [7]:
@model single_force(pos_list, vel_list) = begin
    velocity ~ MvNormal(zeros(2), 10 * ones(2))
    F ~ MvNormal(zeros(2), 10 * ones(2))
    
    obj = Particle(1.0, [0.0, 0.0], velocity)
    env = Earth(Force(F))
    
    for i in 1:length(pos_list)
        obj′ = step(obj, env; dt=0.1)
        pos_list[i] ~ MvNormal(positionof(obj′), 0.1)
        vel_list[i] ~ MvNormal(velocityof(obj′), 0.1)
        obj = obj′
    end
end
chain = sample(single_force(positionof.(traj), velocityof.(traj)), PG(500), 500)

Progress: 100%|█████████████████████████████████████████| Time: 0:02:15


Object of type Chains, with data of type 500×7×1 Array{Float64,3}

Log evidence      = -4787.350058053319
Iterations        = 1:500
Thinning interval = 1
Chains            = 1
Samples per chain = 500
internals         = le, lp, weight
parameters        = F[1], F[2], velocity[1], velocity[2]

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters  │ mean      │ std      │ naive_se  │ mcse     │ ess     │
│     │ Symbol      │ Float64   │ Float64  │ Float64   │ Float64  │ Any     │
├─────┼─────────────┼───────────┼──────────┼───────────┼──────────┼─────────┤
│ 1   │ F[1]        │ -0.461948 │ 3.87942  │ 0.173493  │ 1.36043  │ 5.83467 │
│ 2   │ F[2]        │ -7.91299  │ 4.67508  │ 0.209076  │ 1.4259   │ 4.36312 │
│ 3   │ velocity[1] │ 9.97618   │ 0.986076 │ 0.0440987 │ 0.284643 │ 5.55745 │
│ 4   │ velocity[2] │ 19.5883   │ 1.11058  │ 0.0496667 │ 0.284506 │ 5.96186 │

Quantiles

│ Row │ parameters  │ 2.5%     │ 25.0%    │ 50.0%    │ 75.0%   

In [8]:
# res = get(chain[401:500], [:F])
# res = get(chain[401:500], [:F, :velocity])
# plot(Particle(1.0, [0.0, 0.0], [mean.(res.velocity)...]), Earth(Force([mean.(res.F)...]))) |> display
# plot_data() |> display

In [9]:
using PyCall
pymunk = pyimport.(["pymunk"]);